In [ ]:
!pip install openai

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "..."

In [ ]:
import sqlite3

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# usersテーブルを作成する
cursor.execute('''
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT
)
''')

# productsテーブルを作成する
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price INTEGER
)
''')

# order_historyテーブルを作成する
cursor.execute('''
CREATE TABLE order_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product_id INTEGER,
    purchase_date DATE,
    quantity INTEGER,
    remarks TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

# support_historyテーブルを作成する
cursor.execute('''
CREATE TABLE support_history (
    history_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    subject TEXT,
    message_content TEXT,
    message_type TEXT,
    status TEXT,
    created_at DATETIME,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
)
''')

# 変更をコミットする
conn.commit()

# データベース接続を閉じる
conn.close()

In [ ]:
import sqlite3
from datetime import datetime

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# usersテーブルに日本語のテストデータを挿入する
users_data = [
    (1, '太郎', '山田', 'taro@example.com', '090-1234-5678'),
    (2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')
]

for user in users_data:
    cursor.execute('''
    INSERT INTO users (user_id, first_name, last_name, email, phone)
    VALUES (?, ?, ?, ?, ?)
    ''', user)

# productsテーブルに日本語のテストデータを挿入する
products_data = [
    (1, '商品A', 1000),
    (2, '商品B', 2000),
    (3, '商品C', 3000)
]

for product in products_data:
    cursor.execute('''
    INSERT INTO products (product_id, product_name, price)
    VALUES (?, ?, ?)
    ''', product)

# order_historyテーブルに日本語のテストデータを挿入する
order_history_data = [
    (1, 1, 1, '2023-04-01', 2, '迅速な発送', datetime.now()),
    (2, 2, 3, '2023-04-05', 1, 'ギフトラッピング', datetime.now())
]

for order in order_history_data:
    cursor.execute('''
    INSERT INTO order_history (history_id, user_id, product_id, purchase_date, quantity, remarks, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', order)

# support_historyテーブルに日本語のテストデータを挿入する
support_history_data = [
    (1, 1, '請求に関する問題', None, None, 'open', datetime.now()),
    (2, 1, None, '請求に問題があります。', 'user', None, datetime.now()),
    (3, 1, None, 'お問い合わせいただきありがとうございます。問題を調査しています。', 'support', None, datetime.now())
]

for support in support_history_data:
    cursor.execute('''
    INSERT INTO support_history (history_id, user_id, subject, message_content, message_type, status, created_at)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', support)

# 変更をコミットする
conn.commit()

# データベース接続を閉じる
conn.close()

In [ ]:
import sqlite3

# データベース接続を作成する
conn = sqlite3.connect('user_support.db')

# カーソルオブジェクトを作成する
cursor = conn.cursor()

# 各テーブルからデータを取得して表示する
tables = ['users', 'products', 'order_history', 'support_history']

for table in tables:
    print(f"{table} テーブルのデータ:")
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    print()  # 空行を挿入して見やすくする

# データベース接続を閉じる
conn.close()

users テーブルのデータ:
(1, '太郎', '山田', 'taro@example.com', '090-1234-5678')
(2, '花子', '佐藤', 'hanako@example.com', '080-9876-5432')

products テーブルのデータ:
(1, '商品A', 1000)
(2, '商品B', 2000)
(3, '商品C', 3000)

order_history テーブルのデータ:
(1, 1, 1, '2023-04-01', 2, '迅速な発送', '2023-06-07 13:51:12.365794')
(2, 2, 3, '2023-04-05', 1, 'ギフトラッピング', '2023-06-07 13:51:12.365798')

support_history テーブルのデータ:
(1, 1, '請求に関する問題', None, None, 'open', '2023-06-07 13:51:12.366100')
(2, 1, None, '請求に問題があります。', 'user', None, '2023-06-07 13:51:12.366104')
(3, 1, None, 'お問い合わせいただきありがとうございます。問題を調査しています。', 'support', None, '2023-06-07 13:51:12.366104')



In [ ]:
import openai
import os
import sys
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from sqlalchemy.exc import SQLAlchemyError
from tabulate import tabulate
import json
from json.decoder import JSONDecodeError


engine = create_engine('sqlite:///user_support.db', echo=False)

def init_meta_data():

    # メタデータを作成、テーブル情報を反映させるためにエンジンからテーブル情報を読み込む
    metadata = MetaData()
    metadata.reflect(bind=engine)

    # メタデータのテーブル情報からテーブル名と列情報を編集する
    meta_text = ''
    for table in metadata.tables.values():
        meta_text += 'テーブル名:'+ table.name + '\n'
        for column in table.columns:
            meta_text += f'列名:{column.name}, 型: {column.type}\n'
        meta_text += '\n'
    return meta_text

def exec_sql(sql):
    Session = sessionmaker(bind=engine)

    error_text = "None" 
    rows = []
    tabled = ""
    dict_array = []

    try:
        with Session() as session:
            t = text (sql)
            result = session.execute(t)

            header = [k for k in result.keys()]

            rows = result.fetchall()
            tabled = tabulate(rows,header, tablefmt="plain")
          
            dict_array = []
            for row in rows:
                dict_row = {}
                for i in range(len(header)):
                    dict_row[header[i]] = row[i]
                dict_array.append(dict_row)
            #print(tabled)
            #print()

    except SQLAlchemyError as e:
        print(f'Exception Excute SQL: {e}\n')
        error_text = str(e)

    return {
        "read_count": len(rows),
        "tabled": tabled,
        "dict_array": dict_array,
        "error": error_text
    }

def exec_api(user_text="",user_info="",prompt="",meta_text="",summary="",temperature=0):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature = temperature,
        messages=[
            {
                "role": "user",
                        "content": prompt.format(
                            user_text=user_text,
                            user_info=user_info,
                            prompt=prompt,
                            meta_text=meta_text,
                            summary=summary)
            }
        ],
    )
    return response.choices[0].message.content.strip()


In [ ]:
def find_user(user_text):
  prompt="""
    {{
        "prompt": "
        ## DB情報
        {meta_text}

        ## 処理
        ユーザテーブルを参照し、次の要求文に対してユーザを一意に特定したい。
        その回答(SQL文を含まない)と、SQL文(ID,名前,電話番号,メール)を
        厳密なJSON形式で以下のように提供してください。
        なお、日本語の名前は最初がlast_nameです。
        説明文は不要ですので非表示にしてください。：
          {{
              "response": {{
                "message": "ユーザをDBに問い合わせます)",
                "sql_query": "SQL文"
              }}
          }}

          要求: [{user_text}]
        }}
    }}
  """
  meta_text = init_meta_data() 
  response = exec_api(user_text=user_text,prompt=prompt,meta_text=meta_text)
  #print(response)
  json_answer = json.loads(response)
  sql = json_answer['response']['sql_query']
  if sql:
    result = exec_sql(sql)
    #print(result['read_count'])
    #print(result['tabled'])
    #print(result['dict_array'])
    if result['read_count'] == 1:
      return result['tabled']
    else:
      return ""

if __name__ == "__main__":

  print("お客様の情報を確認します。お名前、電話番号、ユーザID、メールなどお客様を特定できるデータを入力してください。")
  while True:
    user_text = input("\n>")
    user = find_user(user_text)
    print(user)
    if user:
      print("お客様の情報を確認できました。ありがとうございます。")
      break
    else:
      print("お客様の情報を確認できませんでした。再度入力してください。")

お客様の情報を確認します。お名前、電話番号、ユーザID、メールなどお客様を特定できるデータを入力してください。

>ID=1
  user_id  name       phone          email
        1  山田 太郎  090-1234-5678  taro@example.com
お客様の情報を確認できました。ありがとうございます。


In [ ]:
def get_user_info(user):
  prompt="""
  {{
      "prompt": "
      ## DB情報
        {meta_text}

      ## ユーザ情報
        {user_info}

      ## 処理
      DB情報を参照し、
      ユーザ情報をもとにそのユーザに関するすべてがわかるためのタスク
      (購買履歴、対応履歴、購入済商品情報など）を挙げ、
      そのタスクの説明(SQL文を含まない)と、タイトル、SQL文を
      厳密なJSON形式で以下のように提供してください。
      説明文は不要ですので非表示にしてください。：
        {{
            "message":"(回答メッセージ)",
            "queries": [{{
              "title";(タスクのタイトル)
              "description": "(タスクの説明)",
              "sql_query": "(SQL文)"
            }}]
        }}
      
  }}
  """
  meta_text = init_meta_data() 
  user_text = ""
  response = exec_api(user_info=user,prompt=prompt,meta_text=meta_text)
  #print(response)
  json_answer = json.loads(response)
  user_info = "\nユーザ情報\n"+user+"\n"
  for query in json_answer['queries']:
    description = query['description']
    user_info += "\n"+query['title']+"\n"
    #print(title)
    sql = query['sql_query']
    if sql:
      result = exec_sql(sql)
      user_info +=  result['tabled']+"\n"
  return user_info

if __name__ == "__main__":

    print("お客様の情報をDBに問い合わせます。少々お待ちください。")
    user_info = get_user_info(user)
    print(user_info)
    print("お客様の情報をDBから取得できました。")

お客様の情報をDBに問い合わせます。少々お待ちください。

ユーザ情報
  user_id  name       phone          email
        1  山田 太郎  090-1234-5678  taro@example.com

購買履歴の取得
  history_id    user_id    product_id  purchase_date      quantity  remarks     created_at
           1          1             1  2023-04-01                2  迅速な発送  2023-06-07 13:51:12.365794

対応履歴の取得
  history_id    user_id  subject           message_content                                                   message_type    status    created_at
           1          1  請求に関する問題                                                                                    open      2023-06-07 13:51:12.366100
           2          1                    請求に問題があります。                                            user                      2023-06-07 13:51:12.366104
           3          1                    お問い合わせいただきありがとうございます。問題を調査しています。  support                   2023-06-07 13:51:12.366104

購入済商品情報の取得
product_name      price  purchase_date      quantity
商品A           

In [ ]:
user_info

'\nユーザ情報\n  user_id  name       phone          email\n        1  山田 太郎  090-1234-5678  taro@example.com\n\n購買履歴の取得\n  history_id    user_id    product_id  purchase_date      quantity  remarks     created_at\n           1          1             1  2023-04-01                2  迅速な発送  2023-06-07 13:51:12.365794\n\n対応履歴の取得\n  history_id    user_id  subject           message_content                                                   message_type    status    created_at\n           1          1  請求に関する問題                                                                                    open      2023-06-07 13:51:12.366100\n           2          1                    請求に問題があります。                                            user                      2023-06-07 13:51:12.366104\n           3          1                    お問い合わせいただきありがとうございます。問題を調査しています。  support                   2023-06-07 13:51:12.366104\n\n購入済商品情報の取得\nproduct_name      price  purchase_date      quantity\n商品A              1000  20

In [ ]:
def user_support(user_info):
  summary = ""
  prompt="""
    {{
        "prompt": "
        ## 会話の要約
          {summary}
        ## ユーザ情報
          {user_info}

        ## 処理
        あなたは接客のエキスパートです。お客様の要求文にたいして的確に答えてください。
        接客のさい、ユーザ情報を参照し親切丁寧にお客様サポートをしてください。  
        ただし、DB情報にある事実だけを答えてください。     
        お客様からの要求文が「ありがとう。ではまた。さよなら。」などの終了を暗示する言葉が入力された場合サポート終了とみなします。
        回答を含めて以下のように厳密なJSON形式で出力してください。:
          {{
              "message":"(回答メッセージ)",
              "status": "(サポートの状態。開始、サポート中、サポート終了)",
              "summary":"要求文と回答の内容の要約"
          }}

        要求文[{user_text}]
        "
    }}
  """
  while True:
    user_text = input(">")
    response = exec_api(user_text=user_text,user_info=user_info,prompt=prompt,summary=summary,temperature=0.5)
    #print(response)
    json_answer = json.loads(response)
    message = json_answer['message']
    print(f"{message}\n")
    if json_answer['status'] == 'サポート終了':
      return summary
    summary += f"要求: {user_text}\n回答: {json_answer['message']}\n"

if __name__ == "__main__":

    print("お客様、何かご質問やお困り事はございますか？")
    meta_text = init_meta_data() 
    result_summary = user_support(user_info)
    #print(result_summary)
    print("ありがとうございます。それでは失礼いたします。")

In [ ]:
user_info

'\nユーザ情報\n  user_id  name       phone          email\n        1  山田 太郎  090-1234-5678  taro@example.com\n\n購買履歴の取得\n  history_id    user_id    product_id  purchase_date      quantity  remarks     created_at\n           1          1             1  2023-04-01                2  迅速な発送  2023-06-07 13:51:12.365794\n\n対応履歴の取得\n  history_id    user_id  subject           message_content                                                   message_type    status    created_at\n           1          1  請求に関する問題                                                                                    open      2023-06-07 13:51:12.366100\n           2          1                    請求に問題があります。                                            user                      2023-06-07 13:51:12.366104\n           3          1                    お問い合わせいただきありがとうございます。問題を調査しています。  support                   2023-06-07 13:51:12.366104\n\n購入済商品情報の取得\nproduct_name      price  purchase_date      quantity\n商品A              1000  20